In [3]:
import requests
import json
import sys
import pandas as pd
import csv
from  keys  import  client_id, api_key
#from fn import *
# print(sys.path)

In [27]:
url =  'https://api.yelp.com/v3/businesses/search' #points to url of user's yelp developer page

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


# Dynamic: change term, location, and categories to pull different results from call
# * Required for defining url_params
#
# term = 'Film Production'
# location = 'Brooklyn'
# categories = 'Video/Film'
#
# term = 'Film Production'
# location = 'Brooklyn'
# categories = 'Video/Film'
#
term = input('What sort of business do you think you can do? (plumbers, takout, etc.): ')
if term == 'president' or 'President' or 'POTUS':
    print("That's what the last guy said, too.")
location = input('Near where? (please say Monaco): ')
if location != 'Monaco' or 'monaco':
    print('dang I really wanted to work there..')
if location == 'Washington' or 'washington' or 'D.C.' or 'DC':
    print('DC? Suit yourself...')
categories = input('What sort of industry would you like to disrupt?: ')
if 'data' in categories:
    print("don't we all!")
spec = f'{term}_{location}_{categories}_data'
#Note on categories, they appear to be optional. You may search 'All' for all, or possibly leave the field blank.


url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
                "offset": 0
            }

# 
# The variables and requests below are critical, 
# but have been written in to yelp_call() fn
# 
# response = requests.get(url, headers=headers, params=url_params)
# data = response.json()

What sort of business do you think you can do? (plumbers, takout, etc.): Film Equipment Rental
That's what the last guy said, too.
Near where? (please say Monaco): Los Angeles
dang I really wanted to work there..
DC? Suit yourself...
What sort of industry would you like to disrupt?: Video/Film


In [28]:
def yelp_call(headers, url_params):
    """
    This will use the requests module to get from Yelp. 
    What is returned will be modified by our URL parameters.
    This must be called fresh with updated url_params for each call if we want to return more results.
    
    2/14 — defined data variable in this loop.
    """
    response = requests.get(url, headers=headers, params=url_params) # our url, header and params should be consistent, atleast with our Yelp data
    data = response.json()
    return data

In [29]:
term, location, categories

('Film Equipment Rental', 'Los Angeles', 'Video/Film')

In [30]:
def parse_data(list_of_data):
    """
    Input data['businesses'] to return a list of tuples,
    with each tuple containing individual business name, address, and other items
    """
    businesses = []
    for business in list_of_data:
        biz_price = None
        if 'price' not in business.keys():
            biz_price = None
        else:
            biz_price = len(business['price'])
        biz_tuple = (business['name'],
                     business['location']['display_address'],
                     business['location']['city'],
                     business['rating'],
                     business['review_count'],
                     business['coordinates'],
                     biz_price,
                     business['id'],
                     business['categories'])
        businesses.append(biz_tuple)
    return businesses

In [320]:
csv_filepath = f'database/{term}_{location}_database.csv'

def call_1000(csv_filepath):
    """
    JAKE I FUCKING DID IT

    WE DON'T EVEN GET AN ERROR MESSAGE ANYMORE
    """
    url_params['offset'] = 0
    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses']) # list of businesses in tuples
    num = results['total']
    biz_list = []
    while url_params['offset'] < 1000 and len(biz_list) < num:
        for biz in parsed:
            biz_list.append(biz)
        url_params['offset'] += 50
        results = yelp_call(headers, url_params)
        if num >= len(biz_list):
            if 'businesses' not in results:
                break
            else:
                parsed = parse_data(results['businesses']) # list of businesses in tuples
        elif len(biz_list) <= 950:
            continue
        else:
            break
    df = pd.DataFrame(biz_list, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price','Id','Categories'])
    with open(csv_filepath, "a") as f: #'x' creates and writes to csv
        read_file = csv.writer(f)
        df.to_csv(csv_filepath, mode = "a", index = False)
    print('CSV file written to {csv_filepath}.')
    return df

called = call_1000(csv_filepath)
called

CSV file written to {csv_filepath}.


,Name,Address,City,Rating,Review Count,Coordinates,Price,Id,Categories
0,Aire Ancient Baths - New York,"[88 Franklin St, New York, NY 10013]",New York,4.0,840,"{'latitude': 40.71823, 'longitude': -74.00473}",3.0,mM1uQFMZNZtrGHZzB1ghMQ,"[{'alias': 'massage', 'title': 'Massage'}]"
1,Nuansa Spa,"[605 Fifth Ave, Fl 5, New York, NY 10017]",New York,5.0,210,"{'latitude': 40.75755, 'longitude': -73.97752}",3.0,BqNXOy5WJjIoNTX_Xg2wOg,"[{'alias': 'skincare', 'title': 'Skin Care'}, ..."
2,Shibui,"[377 Greenwich St, Manhattan, NY 10013]",Manhattan,4.0,35,"{'latitude': 40.71989, 'longitude': -74.01002}",3.0,ZkI3V9xcL5r8Tw5kS0BgRw,"[{'alias': 'spas', 'title': 'Day Spas'}]"
3,Ohm Spa & Lounge,"[205 E 16th St, Ste M2A (Fl 2), New York, NY 1...",New York,4.5,240,"{'latitude': 40.7345957, 'longitude': -73.9855...",3.0,Nu6403eEBOQvI58_ly2p7w,"[{'alias': 'spas', 'title': 'Day Spas'}, {'ali..."
4,SoJo Spa Club,"[660 River Rd, Edgewater, NJ 07020]",Edgewater,3.5,1667,"{'latitude': 40.81893, 'longitude': -73.97956}",3.0,G4cXJFAnUMoX6BkmCNTbug,"[{'alias': 'hydrotherapy', 'title': 'Hydrother..."
...,...,...,...,...,...,...,...,...,...
995,Jenny Hall Integral Bodywork,"[139 Fulton St, Ste 901, New York, NY 10038]",New York,5.0,18,"{'latitude': 40.7104772, 'longitude': -74.0079...",NaN,Ynjvu-9jAWe5oXiOt5wSng,"[{'alias': 'massage_therapy', 'title': 'Massag..."
996,Healthy Center,"[41-38 College Point Blvd, Queens, NY 11355]",Queens,5.0,1,"{'latitude': 40.75515, 'longitude': -73.83365}",NaN,7keHYHINOPECAHg1x--lmQ,"[{'alias': 'spas', 'title': 'Day Spas'}, {'ali..."
997,Skin Silk,"[1812 Quentin Rd, Ste M2, Brooklyn, NY 11229]",Brooklyn,5.0,21,"{'latitude': 40.60849, 'longitude': -73.95492}",3.0,tPhtgBPblNlFTUDdRJ7FRw,"[{'alias': 'skincare', 'title': 'Skin Care'}, ..."
998,Your Health Success,"[724 Myrtle Ave, Brooklyn, NY 11205]",Brooklyn,4.0,10,"{'latitude': 40.6947, 'longitude': -73.95415}",NaN,QckrJaEF0bmE53XxnHb32w,"[{'alias': 'colonics', 'title': 'Colonics'}, {..."


In [31]:
term_loc = f'database/{term}_{location}_database.csv'
biz_data = pd.read_csv(term_loc)

In [32]:
biz_data

,Name,Address,City,Rating,Review Count,Coordinates,Price,Id,Categories
0,Castex Rentals,"['1044 Cole Ave', 'Los Angeles, CA 90038']",Los Angeles,5.0,29,"{'latitude': 34.0902989663147, 'longitude': -1...",NaN,9ThlqCyvOAPrgedis6Bw0A,"[{'alias': 'videofilmproductions', 'title': 'V..."
1,Lightstone Rentals,"['1441 Gardena Ave', 'Ste 1', 'Glendale, CA 91...",Glendale,5.0,16,"{'latitude': 34.126851, 'longitude': -118.2601...",NaN,9hDT3fdWJoRFkGa62Z0Z6Q,"[{'alias': 'videofilmproductions', 'title': 'V..."
2,Pro HD Rentals,"['2201 Hollywood Way', 'Ste 1', 'Burbank, CA 9...",Burbank,5.0,51,"{'latitude': 34.188555, 'longitude': -118.34928}",2.0,-vpcJktTviMd-K9xX-ZPww,"[{'alias': 'photographystores', 'title': 'Phot..."
3,Filmtools,"['1400 W Burbank Blvd', 'Burbank, CA 91506']",Burbank,3.5,108,"{'latitude': 34.182204, 'longitude': -118.327003}",2.0,J8GxUvGNN1VKj2C7Rfmwcw,"[{'alias': 'photographystores', 'title': 'Phot..."
4,Stray Angel Films,"['11318 Santa Monica Blvd', 'Los Angeles, CA 9...",Los Angeles,4.5,26,"{'latitude': 34.045868, 'longitude': -118.448799}",NaN,QfFgXQiaEg2P7PtcuMie8g,"[{'alias': 'videofilmproductions', 'title': 'V..."
...,...,...,...,...,...,...,...,...,...
115,Hollywood Casting and Film,"['7080 Hollywood Blvd', 'Ste 312', 'Los Angele...",Los Angeles,4.5,119,"{'latitude': 34.1012703829078, 'longitude': -1...",NaN,TqlE1El6dBM8ST58obk4vw,"[{'alias': 'videofilmproductions', 'title': 'V..."
116,Shmily Wedding Photography,"['17891 Sky Park Cir', 'Ste C', 'Irvine, CA 92...",Irvine,5.0,89,"{'latitude': 33.691487, 'longitude': -117.859442}",NaN,Czn5fNHVmjFgVTh0p3jY8w,"[{'alias': 'sessionphotography', 'title': 'Ses..."
117,Assured AudioVisual,"['2941 E Miraloma Ave', 'Ste 3', 'Anaheim, CA ...",Anaheim,4.5,30,"{'latitude': 33.8565903397438, 'longitude': -1...",2.0,tnZf7M7upDqrCqCbxkYM7w,"[{'alias': 'hometheatreinstallation', 'title':..."
118,Something New and Blue Photography Videography,"['130 E Dyer Rd', 'Ste C', 'Santa Ana, CA 92707']",Santa Ana,5.0,56,"{'latitude': 33.7083835, 'longitude': -117.866...",NaN,t60QOLLg2Ad-SLwgJiNkXQ,"[{'alias': 'photographers', 'title': 'Photogra..."


In [33]:
def call_reviews(biz_id): 
    list_of_reviews = []
    for biz in biz_id:
        response = requests.get(f'https://api.yelp.com/v3/businesses/{biz}/reviews',headers = headers)
        review_data = response.json()
        list_of_reviews.append(review_data)
    return list_of_reviews

# list_of_reviews = call_reviews(biz_id)

In [34]:
# call_r = call_reviews(biz_id[:2])
# call_r[:2].keys()

In [35]:
def call_all_reviews(b_data): 
    biz_id = []
    for j in b_data['Id']: #ID column of business data dataframe
        biz_id.append(j)
    list_of_reviews = call_reviews(biz_id)
    return list_of_reviews


In [36]:
def format_reviews(b_data):
    eg = call_all_reviews(b_data)
    list_of_reviews = []
    x = 0
    for i in eg:
        reviews = {}
#         if len(i['reviews']))
        for count in list(range(0, (len(i['reviews'])))):
            reviews[f'Review_{count}'] = i['reviews'][count]['text'] 
        reviews['Id'] = b_data["Id"][x]
        list_of_reviews.append(reviews)
        x+=1
    return list_of_reviews

# format_reviews(biz_data[:20])

In [37]:
def reviews_to_csv(b_data): # If you run for all, this will output IndexError: list index out of range
    csv_filepath = f'database/{term}_{location}_reviews.csv'
    formatted_reviews = format_reviews(b_data)
#     for i in formatted_reviews:
#         if 'reviews' not in i:
#             break
    df = pd.DataFrame(formatted_reviews)
    with open(csv_filepath, "a") as f: #'x' creates and writes to csv
        read_file = csv.writer(f)
        df.to_csv(csv_filepath, mode = "a", index = False)
    return df.head()

reviews_to_csv(biz_data)

,Review_0,Review_1,Review_2,Id
0,"I am a commercial/MV line producer in LA, and ...",I'm a film student and I was very nervous abou...,"Quick, friendly, helpful. Flagged me into thei...",9ThlqCyvOAPrgedis6Bw0A
1,Ryan and the team at Lightstone always make th...,My company and I have only had positive experi...,Highly Highly recommend this place! Lightstone...,9hDT3fdWJoRFkGa62Z0Z6Q
2,Jake at Pro HD Rentals did an amazing job at e...,I was recommended to this place by a very resp...,I would highly recommend Pro HD Rentals for al...,-vpcJktTviMd-K9xX-ZPww
3,"Customer here for years, my go-to shop for any...",Amazing service from Yani this morning. He wa...,The bespectacled 'Barney' behind the counter a...,J8GxUvGNN1VKj2C7Rfmwcw
4,"Quick, easy rental. Stray Angel is willing to ...",I emailed from their website. After 2 days of ...,Always use Stray Angel for my production renta...,QfFgXQiaEg2P7PtcuMie8g


In [115]:
list_of_reviews

[{'Review_0': 'Open at 10am on the dot. About to start taken photos for pluckit magazine. This is a nice studio to work in, got to invest into something like this. \n\n\nWhat...',
  'Review_1': "I booked studio time for a photo shoot last year. It's a GREAT space! Peter was very knowledgeable, helpful, and had everything I needed to help achieve the...",
  'Review_2': "If you're looking for a great photo studio in a hip Brooklyn neighborhood look no further!\n\nIt's a small intimate studio space with a simple cyc wall on the..."},
 {'Review_0': "Hit and Run saved me on a hellish ad campaign! On the last day of the shoot (a monday) they decided to cast a commercial airline pilot who was 6'3 with...",
  'Review_1': "Hit and Run productions saved my shoot! I came to NYC from California. They had some 16mm equipment that I couldn't find available for rental anywhere. Ari...",
  'Review_2': 'H and R filmed in my house, which we lent for an upcoming movie. Extremely professional. Left no mes

In [94]:
for entry in df:
    print(type(entry))


<class 'str'>
<class 'str'>
<class 'str'>


In [76]:
# def call_all_reviews(biz_data): 
    """data = []
    biz_id = []
    for i in biz_data['Id']: #ID column of business data dataframe
        biz_id.append(i)
    list_of_reviews = call_reviews(biz_id)
#         biz_id[0]
    
    for biz in biz_id:
#         print(list_of_reviews[:2])
        reviews = {}
        count = 1
        index = 0
        for i in list_of_reviews:
#             reviews[f'Review_{count}'] = i[f'Review_{count}']['text']
            print(i['reviews'][index]['text'])
            count += 1   
            index +=1
#             reviews['Id'] = biz
        data.append(reviews)
#     csv_filepath = '/database/{term}_reviews
#     with open(csv_filepath, "a") as f: #'x' creates and writes to csv
#         read_file = csv.writer(f)
#         df.to_csv(csv_filepath, mode = "a", index = False)
    return data"""

In [78]:
call_all_reviews(biz_data[:5])

Open at 10am on the dot. About to start taken photos for pluckit magazine. This is a nice studio to work in, got to invest into something like this. 


What...
Hit and Run productions saved my shoot! I came to NYC from California. They had some 16mm equipment that I couldn't find available for rental anywhere. Ari...
Great space and a wonderful group of people to work with.  Helpful, friendly and helpful.  Can't say enough good things.


IndexError: list index out of range

In [14]:
response = requests.get(f'https://api.yelp.com/v3/businesses/{biz_id2[0]}/reviews',headers = headers)
review_data = response.json()

In [26]:
# review_data['reviews'][1]['text'] #list index points to each review 0, 1, 2
# pd.DataFrame(review_data)
review_data

{'reviews': [{'id': 'RskKjU668a_sVDK5-MIzeQ',
   'url': 'https://www.yelp.com/biz/aire-ancient-baths-new-york-new-york?adjust_creative=dCp2ArWfqcaObE2KLuaGww&hrid=RskKjU668a_sVDK5-MIzeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=dCp2ArWfqcaObE2KLuaGww',
   'text': 'Took the chance during Covid and we did not regret it! They take all precautions necessary to ensure safety and the limited appointments allowed by time for...',
   'rating': 5,
   'time_created': '2020-12-06 00:00:24',
   'user': {'id': 'AG2Nbx6T4vGEsmk_Z7TZfA',
    'profile_url': 'https://www.yelp.com/user_details?userid=AG2Nbx6T4vGEsmk_Z7TZfA',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/JZeNPgnyjQnIwCY10QjJzA/o.jpg',
    'name': 'Steph D.'}},
  {'id': 'LQGmtmD09uBpcStvrxv0Vg',
   'url': 'https://www.yelp.com/biz/aire-ancient-baths-new-york-new-york?adjust_creative=dCp2ArWfqcaObE2KLuaGww&hrid=LQGmtmD09uBpcStvrxv0Vg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_sou

In [24]:
for biz in called:
    for review in review_data['review']:
        if biz['Id'] =
        print(review)
        called['Review_1'] = review['text']
called

reviews
total
possible_languages
reviews
total
possible_languages
reviews
total
possible_languages
reviews
total
possible_languages
reviews
total
possible_languages
reviews
total
possible_languages
reviews
total
possible_languages
reviews
total
possible_languages
reviews
total
possible_languages


,Name,Address,City,Rating,Review Count,Coordinates,Price,Id,Categories
0,Urban Myth Studio,"[505 Johnson Ave, Brooklyn, NY 11237]",Brooklyn,5.0,4,"{'latitude': 40.708663918903, 'longitude': -73...",NaN,FOVaV6u49Hb3VEncHJ5eVg,"[{'alias': 'videofilmproductions', 'title': 'V..."
1,Hit and Run Productions,"[65 Roebling St, Ste 202, Brooklyn, NY 11211]",Brooklyn,5.0,4,"{'latitude': 40.7165128, 'longitude': -73.9546...",NaN,ntnuLH-tk9Qh8n895hDV9g,"[{'alias': 'videofilmproductions', 'title': 'V..."
2,Be Electric Studios,"[1298 Willoughby Ave, Brooklyn, NY 11237]",Brooklyn,4.5,4,"{'latitude': 40.7052438, 'longitude': -73.9218...",NaN,lmiG3-1CZFcoQBESPa7KMw,"[{'alias': 'videofilmproductions', 'title': 'V..."
3,Starr Street Studios,"[207-A Starr St, Ste 3-1, Brooklyn, NY 11237]",Brooklyn,5.0,8,"{'latitude': 40.705122, 'longitude': -73.923457}",NaN,rXjhzma0XIrxJEFt7q0tIg,"[{'alias': 'videofilmproductions', 'title': 'V..."
4,Morris-Jumel Mansion,"[65 Jumel Ter, New York, NY 10032]",New York,4.5,38,"{'latitude': 40.83449, 'longitude': -73.93851}",NaN,Sn9jYOKrk0CgeZVGzKAJPw,"[{'alias': 'museums', 'title': 'Museums'}, {'a..."
...,...,...,...,...,...,...,...,...,...
995,Envision Video Services,"[224 Blvd, Hasbrouck Heights, NJ 07604]",Hasbrouck Heights,4.5,9,"{'latitude': 40.8621579820709, 'longitude': -7...",NaN,lH825qSuWbV9ltiZjP-L6Q,"[{'alias': 'videofilmproductions', 'title': 'V..."
996,I Milky 沐白,"[992 6th Ave, New York, NY 10018]",New York,4.5,137,"{'latitude': 40.7513413, 'longitude': -73.9864...",1.0,FKjAriOhhGZJSlkqrk8zWg,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}..."
997,Wheated,"[905 Church Ave, Brooklyn, NY 11218]",Brooklyn,4.5,335,"{'latitude': 40.6465968960541, 'longitude': -7...",2.0,_AqeaDQnMaErG19LobuU8Q,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias..."
998,Amsterdam Ale House,"[340 Amsterdam Ave, Ste 1, New York, NY 10024]",New York,3.0,378,"{'latitude': 40.7814117451455, 'longitude': -7...",2.0,o-FxsaWAM3JwMMqxQb3MbQ,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':..."


In [63]:
what_i_looks_like = call_reviews(biz_id2)

In [66]:
what_i_looks_like[0]

{'Review_1': 'Took the chance during Covid and we did not regret it! They take all precautions necessary to ensure safety and the limited appointments allowed by time for...',
 'Id': 'mM1uQFMZNZtrGHZzB1ghMQ',
 'Review_2': 'Definitely something everyone should experience in their lifetime! I was blown away at the entire experience. Staff lets you pre check in before your visit...',
 'Review_3': 'I really enjoyed my solo-experience with Aire. I only did the thermal bath experience, but I felt entirely relaxed after the 90 minutes of dipping into...'}

In [69]:
spa_data = pd.read_csv('database/spa_NYC_database.csv')

In [73]:
def biz_review_merge():
    for i in what_i_looks_like:
        if spa_data['Id'] == i['Id']:
            spa_data['Review_1'] = i['Review_1']
            spa_data['Review_2'] = i['Review_2']
            spa_data['Review_3'] = i['Review_3']
    return spa_data

In [74]:
biz_review_merge()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [70]:
spa_data

,Name,Address,City,Rating,Review Count,Coordinates,Price,Id
0,Aire Ancient Baths - New York,"['88 Franklin St', 'New York, NY 10013']",New York,4.0,840,"{'latitude': 40.71823, 'longitude': -74.00473}",3.0,mM1uQFMZNZtrGHZzB1ghMQ
1,Nuansa Spa,"['605 Fifth Ave', 'Fl 5', 'New York, NY 10017']",New York,5.0,209,"{'latitude': 40.75755, 'longitude': -73.97752}",3.0,BqNXOy5WJjIoNTX_Xg2wOg
2,Shibui,"['377 Greenwich St', 'Manhattan, NY 10013']",Manhattan,4.0,35,"{'latitude': 40.71989, 'longitude': -74.01002}",3.0,ZkI3V9xcL5r8Tw5kS0BgRw
3,Ohm Spa & Lounge,"['205 E 16th St', 'Ste M2A (Fl 2)', 'New York,...",New York,4.5,240,"{'latitude': 40.7345957, 'longitude': -73.9855...",3.0,Nu6403eEBOQvI58_ly2p7w
4,SoJo Spa Club,"['660 River Rd', 'Edgewater, NJ 07020']",Edgewater,3.5,1667,"{'latitude': 40.81893, 'longitude': -73.97956}",3.0,G4cXJFAnUMoX6BkmCNTbug
...,...,...,...,...,...,...,...,...
995,Dominiques Travel Spa,"['Oceanside, NY 11572']",Oceanside,5.0,2,"{'latitude': 40.637088775634766, 'longitude': ...",NaN,sOwJ1x8Xcte5ehkfo8TA_Q
996,Satori Laser ® Brooklyn Laser Hair Removal,"['155 Montague St', 'New York, NY 11201']",New York,5.0,17,"{'latitude': 40.6946808, 'longitude': -73.993156}",NaN,a0oONwgvDCCjtBBvDlEt7g
997,Tangles Salon & Spa,"['1722 Sheepshead Bay Rd', 'Brooklyn, NY 11235']",Brooklyn,4.0,28,"{'latitude': 40.58554, 'longitude': -73.95174}",3.0,0Ig2273ZKnYjx9pgtZSMBA
998,Spavia Clark Commons,"['1255 Raritan Rd', 'Clark, NJ 07066']",Clark,5.0,1,"{'latitude': 40.63172, 'longitude': -74.30488}",NaN,WCaUuKkQKNHTGUm-1U4eRQ


In [58]:
list_of_reviews#[0]['reviews'][0]['text']

[{'reviews': [{'id': 'RskKjU668a_sVDK5-MIzeQ',
    'url': 'https://www.yelp.com/biz/aire-ancient-baths-new-york-new-york?adjust_creative=dCp2ArWfqcaObE2KLuaGww&hrid=RskKjU668a_sVDK5-MIzeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=dCp2ArWfqcaObE2KLuaGww',
    'text': 'Took the chance during Covid and we did not regret it! They take all precautions necessary to ensure safety and the limited appointments allowed by time for...',
    'rating': 5,
    'time_created': '2020-12-06 00:00:24',
    'user': {'id': 'AG2Nbx6T4vGEsmk_Z7TZfA',
     'profile_url': 'https://www.yelp.com/user_details?userid=AG2Nbx6T4vGEsmk_Z7TZfA',
     'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/JZeNPgnyjQnIwCY10QjJzA/o.jpg',
     'name': 'Steph D.'}},
   {'id': 'LQGmtmD09uBpcStvrxv0Vg',
    'url': 'https://www.yelp.com/biz/aire-ancient-baths-new-york-new-york?adjust_creative=dCp2ArWfqcaObE2KLuaGww&hrid=LQGmtmD09uBpcStvrxv0Vg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_revi

In [ ]:
# FIGURE OUT LATER HOW TO ITERATE THROUGH ALL URL REVIEWS AND ADD TO BUSINESS / LOCATION SPECIFIC DATAFRAME
def call_all_reviews():
    data = []
    for biz in called:
    for review in review_data['review']:
        if biz['Id'] =
        print(review)
        called['Review_1'] = review['text']
    
    for biz in biz_id2:
        response = requests.get(f'https://api.yelp.com/v3/businesses/{biz}/reviews',headers = headers)
        review_data = response.json()
        data.append(review_data)
    return data

In [ ]:
call_reviews(called)

In [69]:
biz_reviews = call_reviews()

In [74]:
biz_reviews[0]

{'reviews': [{'id': 'RskKjU668a_sVDK5-MIzeQ',
   'url': 'https://www.yelp.com/biz/aire-ancient-baths-new-york-new-york?adjust_creative=dCp2ArWfqcaObE2KLuaGww&hrid=RskKjU668a_sVDK5-MIzeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=dCp2ArWfqcaObE2KLuaGww',
   'text': 'Took the chance during Covid and we did not regret it! They take all precautions necessary to ensure safety and the limited appointments allowed by time for...',
   'rating': 5,
   'time_created': '2020-12-06 00:00:24',
   'user': {'id': 'AG2Nbx6T4vGEsmk_Z7TZfA',
    'profile_url': 'https://www.yelp.com/user_details?userid=AG2Nbx6T4vGEsmk_Z7TZfA',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/JZeNPgnyjQnIwCY10QjJzA/o.jpg',
    'name': 'Steph D.'}},
  {'id': 'LQGmtmD09uBpcStvrxv0Vg',
   'url': 'https://www.yelp.com/biz/aire-ancient-baths-new-york-new-york?adjust_creative=dCp2ArWfqcaObE2KLuaGww&hrid=LQGmtmD09uBpcStvrxv0Vg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_sou

In [32]:
spa_

,Name,Address,City,Rating,Review Count,Coordinates,Price,Id
0,Aire Ancient Baths - New York,"['88 Franklin St', 'New York, NY 10013']",New York,4.0,840,"{'latitude': 40.71823, 'longitude': -74.00473}",3.0,mM1uQFMZNZtrGHZzB1ghMQ
1,Nuansa Spa,"['605 Fifth Ave', 'Fl 5', 'New York, NY 10017']",New York,5.0,209,"{'latitude': 40.75755, 'longitude': -73.97752}",3.0,BqNXOy5WJjIoNTX_Xg2wOg
2,Shibui,"['377 Greenwich St', 'Manhattan, NY 10013']",Manhattan,4.0,35,"{'latitude': 40.71989, 'longitude': -74.01002}",3.0,ZkI3V9xcL5r8Tw5kS0BgRw
3,Ohm Spa & Lounge,"['205 E 16th St', 'Ste M2A (Fl 2)', 'New York,...",New York,4.5,240,"{'latitude': 40.7345957, 'longitude': -73.9855...",3.0,Nu6403eEBOQvI58_ly2p7w
4,SoJo Spa Club,"['660 River Rd', 'Edgewater, NJ 07020']",Edgewater,3.5,1667,"{'latitude': 40.81893, 'longitude': -73.97956}",3.0,G4cXJFAnUMoX6BkmCNTbug
...,...,...,...,...,...,...,...,...
995,Dominiques Travel Spa,"['Oceanside, NY 11572']",Oceanside,5.0,2,"{'latitude': 40.637088775634766, 'longitude': ...",NaN,sOwJ1x8Xcte5ehkfo8TA_Q
996,Satori Laser ® Brooklyn Laser Hair Removal,"['155 Montague St', 'New York, NY 11201']",New York,5.0,17,"{'latitude': 40.6946808, 'longitude': -73.993156}",NaN,a0oONwgvDCCjtBBvDlEt7g
997,Tangles Salon & Spa,"['1722 Sheepshead Bay Rd', 'Brooklyn, NY 11235']",Brooklyn,4.0,28,"{'latitude': 40.58554, 'longitude': -73.95174}",3.0,0Ig2273ZKnYjx9pgtZSMBA
998,Spavia Clark Commons,"['1255 Raritan Rd', 'Clark, NJ 07066']",Clark,5.0,1,"{'latitude': 40.63172, 'longitude': -74.30488}",NaN,WCaUuKkQKNHTGUm-1U4eRQ


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Tests below 
### (first cell through yelp call must be run before these)

In [108]:
yelp_call(headers, url_params)

{'businesses': [{'id': 'CwBYC8nVLbG7KjVU6nwR4A',
   'alias': 'dijifi-brooklyn',
   'name': 'DiJiFi',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/rzH6f1btqYDEXZtrCa_uFg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/dijifi-brooklyn?adjust_creative=vGqQbkXzrgSl7z2qib6lAg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vGqQbkXzrgSl7z2qib6lAg',
   'review_count': 165,
   'categories': [{'alias': 'photographystores',
     'title': 'Photography Stores & Services'},
    {'alias': 'videofilmproductions', 'title': 'Video/Film Production'},
    {'alias': 'digitizingservices', 'title': 'Digitizing Services'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.7382584, 'longitude': -73.9549479},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '1166 Manhattan Ave',
    'address2': 'Ste 201',
    'address3': '',
    'city': 'Brooklyn',
    'zip_code': '11222',
    'country': 'US',
    'state': 'NY',
    'display_address': 

In [117]:
# data['total']
yelp_call(headers, url_params)['total']

298

#### need to run function parse_data() between these cells

In [73]:
parsed = parse_data(data['businesses'])
parsed

[('DiJiFi',
  ['1166 Manhattan Ave', 'Ste 201', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  165,
  {'latitude': 40.7382584, 'longitude': -73.9549479},
  2),
 ('590films',
  ['42 West St', 'Ste 301', 'Brooklyn, NY 11222'],
  'Brooklyn',
  5.0,
  4,
  {'latitude': 40.7280427, 'longitude': -73.9588363},
  None),
 ("Shell's Loft Brooklyn",
  ['13 Lexington Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  5.0,
  2,
  {'latitude': 40.6866128, 'longitude': -73.9616521},
  None),
 ('Mind of Who Productions',
  ['193 Marcus Garvey Blvd', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.691765, 'longitude': -73.939434},
  None),
 ('Kaleidoscope Pictures',
  ['Brooklyn, NY 11231'],
  'Brooklyn',
  5.0,
  8,
  {'latitude': 40.6772802, 'longitude': -74.0094471},
  None),
 ('Cinema World Studios',
  ['220 Dupont St', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.735954, 'longitude': -73.950508},
  None),
 ('Starr Street Media',
  ['852 Rogers Ave', 'Brooklyn, NY 

In [74]:
pd.DataFrame(parsed, columns =['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN
3,Mind of Who Productions,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,4.5,2,"{'latitude': 40.691765, 'longitude': -73.939434}",NaN
4,Kaleidoscope Pictures,"[Brooklyn, NY 11231]",Brooklyn,5.0,8,"{'latitude': 40.6772802, 'longitude': -74.0094...",NaN
5,Cinema World Studios,"[220 Dupont St, Brooklyn, NY 11222]",Brooklyn,4.5,2,"{'latitude': 40.735954, 'longitude': -73.950508}",NaN
6,Starr Street Media,"[852 Rogers Ave, Brooklyn, NY 11226]",Brooklyn,5.0,1,"{'latitude': 40.64986, 'longitude': -73.95257}",NaN
7,Hit and Run Productions,"[65 Roebling St, Ste 202, Brooklyn, NY 11211]",Brooklyn,5.0,4,"{'latitude': 40.7165128, 'longitude': -73.9546...",NaN
8,BK Actors Self-Tape,"[141 Flushing Ave, Bldg 77 ,Ste 1336, Brooklyn...",Brooklyn,5.0,14,"{'latitude': 40.6985551, 'longitude': -73.9712...",NaN
9,Bizvidmedia Productions,"[99 Wallstreet, New York, NY 10005]",New York,5.0,5,"{'latitude': 40.70479, 'longitude': -74.00726}",NaN


In [53]:
parsed

[('DiJiFi',
  ['1166 Manhattan Ave', 'Ste 201', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  165,
  {'latitude': 40.7382584, 'longitude': -73.9549479},
  2),
 ('590films',
  ['42 West St', 'Ste 301', 'Brooklyn, NY 11222'],
  'Brooklyn',
  5.0,
  4,
  {'latitude': 40.7280427, 'longitude': -73.9588363},
  None),
 ("Shell's Loft Brooklyn",
  ['13 Lexington Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  5.0,
  2,
  {'latitude': 40.6866128, 'longitude': -73.9616521},
  None),
 ('Mind of Who Productions',
  ['193 Marcus Garvey Blvd', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.691765, 'longitude': -73.939434},
  None),
 ('Kaleidoscope Pictures',
  ['Brooklyn, NY 11231'],
  'Brooklyn',
  5.0,
  8,
  {'latitude': 40.6772802, 'longitude': -74.0094471},
  None),
 ('Cinema World Studios',
  ['220 Dupont St', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.735954, 'longitude': -73.950508},
  None),
 ('Starr Street Media',
  ['852 Rogers Ave', 'Brooklyn, NY 

In [54]:
type(len(df))

int

In [72]:
df = pd.DataFrame(data, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
results = yelp_call(headers, url_params)
parsed = parse_data(results['businesses'])
df_2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
df.append(df_2, ignore_index=True)
df

,Name,Address,City,Rating,Review Count,Coordinates,Price


### Testing for call_1000

In [72]:
df = pd.DataFrame(data, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
results = yelp_call(headers, url_params)
parsed = parse_data(results['businesses'])
df_2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
df.append(df_2, ignore_index=True)
df

,Name,Address,City,Rating,Review Count,Coordinates,Price


In [ ]:
csv_filepath = f'database/{term}_{location}_database.csv'

In [ ]:
current_offset = 0
url_params['offset'] = current_offset
results = yelp_call(headers, url_params)
parsed = parse_data(results['businesses'])
num = results['total']
df = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
while current_offset < (num+50) and current_offset < 1000:     

    
#         concat or append df here
#             current_offset += 50
#             url_params['offset'] = current_offset
#             results = yelp_call(headers, url_params)
#             parsed = parse_data(results['businesses'])

        df2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
        df.append(df2)
    current_offset += 50
    url_params['offset'] = current_offset
    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses'])
return df

In [71]:
#testing call_1000() contents outside of function

current_offset = url_params['offset']
df = pd.DataFrame(data, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
num = 0
while current_offset < (num+1) and current_offset < 951: 
    url_params['offset'] = current_offset

    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses'])
    #concat or append df here
    df_2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
    df.append(df_2, ignore_index=True)
#     num += len(df)
#     current_offset += 50
#     url_params['offset'] = current_offset
    print(df, current_offset, num, url_params)

Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categori

KeyboardInterrupt: 

In [29]:
pd.DataFrame(data)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [103]:
# testing DataFrame output from parsed (parsed = parse_data(data['businesses']))

pd.DataFrame(parsed[0:3], columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN


In [117]:
# data['total']
yelp_call(headers, url_params)['total']

298

In [ ]:
# df = pd.DataFrame(parsed_results, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

In [ ]:
# your code to open the csv file, concat the current data, and save the data. 

def df_save(csv_filepath, df): # MIKE NOTE: an if statement for if the file exists may be useful
    with open(csv_filepath, "a") as f: #'x' creates and writes to csv
        read_file = csv.writer(f)
        df.to_csv(csv_filepath, mode = "a", index = False)
# this will append to csv. 2nd arg index=False to not save the inde

In [ ]:
df_save(f'database/{term}_{location}_database.csv', df)

In [71]:
current_offset = url_params['offset']
df = pd.DataFrame(data, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
num = 0
while current_offset < (num+1) and current_offset < 951: 
    url_params['offset'] = current_offset

    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses'])
    #concat or append df here
    df_2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
    df.append(df_2, ignore_index=True)
#     num += len(df)
#     current_offset += 50
#     url_params['offset'] = current_offset
    print(df, current_offset, num, url_params)

Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categori

KeyboardInterrupt: 

In [73]:
parsed = parse_data(data['businesses'])
parsed

[('DiJiFi',
  ['1166 Manhattan Ave', 'Ste 201', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  165,
  {'latitude': 40.7382584, 'longitude': -73.9549479},
  2),
 ('590films',
  ['42 West St', 'Ste 301', 'Brooklyn, NY 11222'],
  'Brooklyn',
  5.0,
  4,
  {'latitude': 40.7280427, 'longitude': -73.9588363},
  None),
 ("Shell's Loft Brooklyn",
  ['13 Lexington Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  5.0,
  2,
  {'latitude': 40.6866128, 'longitude': -73.9616521},
  None),
 ('Mind of Who Productions',
  ['193 Marcus Garvey Blvd', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.691765, 'longitude': -73.939434},
  None),
 ('Kaleidoscope Pictures',
  ['Brooklyn, NY 11231'],
  'Brooklyn',
  5.0,
  8,
  {'latitude': 40.6772802, 'longitude': -74.0094471},
  None),
 ('Cinema World Studios',
  ['220 Dupont St', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.735954, 'longitude': -73.950508},
  None),
 ('Starr Street Media',
  ['852 Rogers Ave', 'Brooklyn, NY 

In [74]:
pd.DataFrame(parsed, columns =['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN
3,Mind of Who Productions,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,4.5,2,"{'latitude': 40.691765, 'longitude': -73.939434}",NaN
4,Kaleidoscope Pictures,"[Brooklyn, NY 11231]",Brooklyn,5.0,8,"{'latitude': 40.6772802, 'longitude': -74.0094...",NaN
5,Cinema World Studios,"[220 Dupont St, Brooklyn, NY 11222]",Brooklyn,4.5,2,"{'latitude': 40.735954, 'longitude': -73.950508}",NaN
6,Starr Street Media,"[852 Rogers Ave, Brooklyn, NY 11226]",Brooklyn,5.0,1,"{'latitude': 40.64986, 'longitude': -73.95257}",NaN
7,Hit and Run Productions,"[65 Roebling St, Ste 202, Brooklyn, NY 11211]",Brooklyn,5.0,4,"{'latitude': 40.7165128, 'longitude': -73.9546...",NaN
8,BK Actors Self-Tape,"[141 Flushing Ave, Bldg 77 ,Ste 1336, Brooklyn...",Brooklyn,5.0,14,"{'latitude': 40.6985551, 'longitude': -73.9712...",NaN
9,Bizvidmedia Productions,"[99 Wallstreet, New York, NY 10005]",New York,5.0,5,"{'latitude': 40.70479, 'longitude': -74.00726}",NaN


In [53]:
parsed

[('DiJiFi',
  ['1166 Manhattan Ave', 'Ste 201', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  165,
  {'latitude': 40.7382584, 'longitude': -73.9549479},
  2),
 ('590films',
  ['42 West St', 'Ste 301', 'Brooklyn, NY 11222'],
  'Brooklyn',
  5.0,
  4,
  {'latitude': 40.7280427, 'longitude': -73.9588363},
  None),
 ("Shell's Loft Brooklyn",
  ['13 Lexington Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  5.0,
  2,
  {'latitude': 40.6866128, 'longitude': -73.9616521},
  None),
 ('Mind of Who Productions',
  ['193 Marcus Garvey Blvd', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.691765, 'longitude': -73.939434},
  None),
 ('Kaleidoscope Pictures',
  ['Brooklyn, NY 11231'],
  'Brooklyn',
  5.0,
  8,
  {'latitude': 40.6772802, 'longitude': -74.0094471},
  None),
 ('Cinema World Studios',
  ['220 Dupont St', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.735954, 'longitude': -73.950508},
  None),
 ('Starr Street Media',
  ['852 Rogers Ave', 'Brooklyn, NY 

In [54]:
type(len(df))

int

In [204]:
called_1k = call_1000()

In [127]:
parsed
test_d = pd.DataFrame
test_d2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
pd.concat([test_d, parsed])
test_d
type(parsed)

TypeError: _consolidate() missing 1 required positional argument: 'self'

In [205]:
called_1k.iloc[45:60]

,Name,Address,City,Rating,Review Count,Coordinates,Price
45,Brain Fry Productions,"[2853 Steinway St, 2F, Astoria, NY 11103]",Astoria,5.0,2,"{'latitude': 40.630508, 'longitude': -74.028456}",NaN
46,Silvercup Studios,"[42-22 22nd St, Long Island City, NY 11101]",Long Island City,4.5,12,"{'latitude': 40.751331, 'longitude': -73.943515}",NaN
47,YouTube Space,"[75 9th Ave, New York, NY 10011]",New York,5.0,6,"{'latitude': 40.742581, 'longitude': -74.005434}",NaN
48,Sun Production,"[36-06 30th St, C7, Astoria, NY 11106]",Astoria,5.0,1,"{'latitude': 40.756815, 'longitude': -73.930988}",NaN
49,Broadway Stages,"[359 Kingsland Ave, Greenpoint, NY 11222]",Greenpoint,4.0,1,"{'latitude': 40.7289565103425, 'longitude': -7...",NaN


In [163]:
test_parsed = parse_data(results['businesses'])
test_parsed
test_d = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
test_d.append(parsed)

,0,1,2,3,4,5,6,Address,City,Coordinates,Name,Price,Rating,Review Count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,"{'latitude': 40.7382584, 'longitude': -73.9549...",DiJiFi,2.0,4.5,165.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,"{'latitude': 40.7280427, 'longitude': -73.9588...",590films,NaN,5.0,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,"{'latitude': 40.6866128, 'longitude': -73.9616...",Shell's Loft Brooklyn,NaN,5.0,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,"{'latitude': 40.691765, 'longitude': -73.939434}",Mind of Who Productions,NaN,4.5,2.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Brooklyn, NY 11231]",Brooklyn,"{'latitude': 40.6772802, 'longitude': -74.0094...",Kaleidoscope Pictures,NaN,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Brain Fry Productions,"[2853 Steinway St, 2F, Astoria, NY 11103]",Astoria,5.0,2.0,"{'latitude': 40.630508, 'longitude': -74.028456}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Silvercup Studios,"[42-22 22nd St, Long Island City, NY 11101]",Long Island City,4.5,12.0,"{'latitude': 40.751331, 'longitude': -73.943515}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,YouTube Space,"[75 9th Ave, New York, NY 10011]",New York,5.0,6.0,"{'latitude': 40.742581, 'longitude': -74.005434}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,Sun Production,"[36-06 30th St, C7, Astoria, NY 11106]",Astoria,5.0,1.0,"{'latitude': 40.756815, 'longitude': -73.930988}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
df = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
df

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN
3,Mind of Who Productions,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,4.5,2,"{'latitude': 40.691765, 'longitude': -73.939434}",NaN
4,Kaleidoscope Pictures,"[Brooklyn, NY 11231]",Brooklyn,5.0,8,"{'latitude': 40.6772802, 'longitude': -74.0094...",NaN
5,Cinema World Studios,"[220 Dupont St, Brooklyn, NY 11222]",Brooklyn,4.5,2,"{'latitude': 40.735954, 'longitude': -73.950508}",NaN
6,Starr Street Media,"[852 Rogers Ave, Brooklyn, NY 11226]",Brooklyn,5.0,1,"{'latitude': 40.64986, 'longitude': -73.95257}",NaN
7,Hit and Run Productions,"[65 Roebling St, Ste 202, Brooklyn, NY 11211]",Brooklyn,5.0,4,"{'latitude': 40.7165128, 'longitude': -73.9546...",NaN
8,BK Actors Self-Tape,"[141 Flushing Ave, Bldg 77 ,Ste 1336, Brooklyn...",Brooklyn,5.0,14,"{'latitude': 40.6985551, 'longitude': -73.9712...",NaN
9,Bizvidmedia Productions,"[99 Wallstreet, New York, NY 10005]",New York,5.0,5,"{'latitude': 40.70479, 'longitude': -74.00726}",NaN


In [206]:
called_1k.tail()

,Name,Address,City,Rating,Review Count,Coordinates,Price
45,Brain Fry Productions,"[2853 Steinway St, 2F, Astoria, NY 11103]",Astoria,5.0,2,"{'latitude': 40.630508, 'longitude': -74.028456}",NaN
46,Silvercup Studios,"[42-22 22nd St, Long Island City, NY 11101]",Long Island City,4.5,12,"{'latitude': 40.751331, 'longitude': -73.943515}",NaN
47,YouTube Space,"[75 9th Ave, New York, NY 10011]",New York,5.0,6,"{'latitude': 40.742581, 'longitude': -74.005434}",NaN
48,Sun Production,"[36-06 30th St, C7, Astoria, NY 11106]",Astoria,5.0,1,"{'latitude': 40.756815, 'longitude': -73.930988}",NaN
49,Broadway Stages,"[359 Kingsland Ave, Greenpoint, NY 11222]",Greenpoint,4.0,1,"{'latitude': 40.7289565103425, 'longitude': -7...",NaN


In [163]:
test_parsed = parse_data(results['businesses'])
test_parsed
test_d = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
test_d.append(parsed)

,0,1,2,3,4,5,6,Address,City,Coordinates,Name,Price,Rating,Review Count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,"{'latitude': 40.7382584, 'longitude': -73.9549...",DiJiFi,2.0,4.5,165.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,"{'latitude': 40.7280427, 'longitude': -73.9588...",590films,NaN,5.0,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,"{'latitude': 40.6866128, 'longitude': -73.9616...",Shell's Loft Brooklyn,NaN,5.0,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,"{'latitude': 40.691765, 'longitude': -73.939434}",Mind of Who Productions,NaN,4.5,2.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Brooklyn, NY 11231]",Brooklyn,"{'latitude': 40.6772802, 'longitude': -74.0094...",Kaleidoscope Pictures,NaN,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Brain Fry Productions,"[2853 Steinway St, 2F, Astoria, NY 11103]",Astoria,5.0,2.0,"{'latitude': 40.630508, 'longitude': -74.028456}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Silvercup Studios,"[42-22 22nd St, Long Island City, NY 11101]",Long Island City,4.5,12.0,"{'latitude': 40.751331, 'longitude': -73.943515}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,YouTube Space,"[75 9th Ave, New York, NY 10011]",New York,5.0,6.0,"{'latitude': 40.742581, 'longitude': -74.005434}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,Sun Production,"[36-06 30th St, C7, Astoria, NY 11106]",Astoria,5.0,1.0,"{'latitude': 40.756815, 'longitude': -73.930988}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [336]:
len(biz_list)

296

In [337]:
num
# len(biz_list)

298

In [341]:
df.iloc[198:202]

,Name,Address,City,Rating,Review Count,Coordinates,Price
198,Musiclab NYC,"[Elmont, NY 11003]",Elmont,5.0,1,"{'latitude': 40.6943581912167, 'longitude': -7...",NaN
199,Crystal Lake Pictures,"[101 Crawfords Corner Rd, Ste 4-101R, Holmdel,...",Holmdel,3.0,2,"{'latitude': 40.3650877, 'longitude': -74.167457}",NaN
200,Dancejock Productions,"[40 Burt Dr, Ste 1, Deer Park, NY 11729]",Deer Park,4.5,12,"{'latitude': 40.758861, 'longitude': -73.305488}",NaN
201,rjephoto,"[basking ridge, NJ 07920]",basking ridge,5.0,2,"{'latitude': 40.7064399719238, 'longitude': -7...",NaN


In [288]:
url_params

{'term': 'Film+Production',
 'location': 'Brooklyn',
 'categories': 'Video/Film',
 'limit': 50,
 'offset': 0}

In [246]:
"""
GOD FUCKING DAMMIT
"""
df.iloc[48:52]

,Name,Address,City,Rating,Review Count,Coordinates,Price
48,Steve Palmer Films,"[New York, NY 11215]",New York,5.0,1,"{'latitude': 40.66874, 'longitude': -73.98181}",NaN
49,Prosper Digital TV,"[33 35th St, Fl 5. Ste 523, Brooklyn, NY 11232]",Brooklyn,5.0,6,"{'latitude': 40.65722, 'longitude': -74.00735}",NaN
50,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
51,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN


In [241]:
# len(biz_list) ## 1000

TypeError: unhashable type: 'list'

In [228]:
yelp_call(headers, url_params)

{'error': {'code': 'VALIDATION_ERROR',
  'description': 'Too many results requested, limit+offset must be <= 1000.'}}

In [231]:
len(biz_list)

0

In [ ]:
def get_restaurants(city, offset):
    """
    THIS IS ANOTHER APPROACH FROM GITHUB
    Make API request to Yelp to get restaurants for a city, and offset the results by an amount.

    Note that Yelp only returns 20 results each time, which is why we need to offset if we want
    the next Nth results.
    """

      """ WE HAVE THIS """
#     # Read Yelp API keys
#     with io.open('config_secret.json') as cred:
#         creds = json.load(cred)
#         auth = Oauth1Authenticator(**creds)
#         client = Client(auth)

    # Set search parameters for Yelp API request
    # Set term as restaurant to get restaurants back as the results
    # Also pass in offset, so Yelp knows how much to offset by

      """ WE HAVE THIS TOO """
    #     params = {
#         'term': 'restaurant',
#         'offset': offset
#     }

    # Make Yelp API call and return the API response
    return client.search(city, **params)

In [113]:
results = yelp_call(headers, url_params)
parsed = parse_data(results['businesses'])
len(parsed)

50

In [ ]:
 """ 
 THIS IS PART 2 OF ANOTHER APPROACH FROM GITHUB
 Get all restaurants for a city from Yelp and load restaurants into database.
 """

    # Get city id, as city id is a required parameter when adding a restaurant to the database
    city_id = get_city_id(city)

    # Start offset at 0 to return the first 20 results from Yelp API request
    offset = 0
    response = get_restaurants(city, offset)

    # Get total number of restaurants for this city
    total_results = response.total 
    """ 
    ^ AHA, THIS IS WHAT WE NEED FOR num IN OUR WHILE LOOP!
    
    JAKE POINTED TO THIS BUT INPUT TOTAL OF DATA VARIABLE, NOT JSON TOTAL
    
    WOULD BE EQUIVALENT TO LEN(PARSED) RIGHT AFTER DEFINING PARSED IN OUR WHILE LOOP
    
    PROBABLY DO NOT NEED IT
    ACTUALLY ENSURING THAT WE ARE NOT CALLING MORE THAN TOTAL POSSIBLE RESULTS 
    (AS JAKE WAS DOING WITH DATA['TOTAL'] IS SMART
    """

    # Offset by 20 each time to get all restaurants and load each restaurant into the database
    while offset < total_results:

        #CODE BELOW is essentially our PARSE_DATA() function
        for business in response.businesses:
            restaurant = Restaurant(city_id=city_id,
                                    name=business.name,
                                    address=" ".join(business.location.display_address),
                                    phone=business.display_phone,
                                    image_url=business.image_url,
                                    latitude=business.location.coordinate.latitude,
                                    longitude=business.location.coordinate.longitude)

            # Add each restaurant to the db
        #THIS IS WHAT WE NEED TO DO CORRECTLY
            db.session.add(restaurant)

        offset += 20

        response = get_restaurants(city, offset)


In [103]:
pd.DataFrame(parsed[0:3], columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN


In [203]:
def call_1000():
    """ MIKE NOTES: 
    12/13 — I believe num should be referring to the number of 
    results that are in our dataframe or dict.
    
    12/14 — Might not be a bad idea to loop through offset BEFORE writing to a csv file.
    This could be done by offsetting, adding new to a dict, then writing to a csv file
    *afterwards outside of our while-loop*. 
    ***Persistent opening/writing to csv is messy AF***
    
    df might need to be defined BEFORE while-loop, then APPENDED or CONCAT within while-loop.
    """
    current_offset = 0
    url_params['offset'] = current_offset
    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses'])
    num = results['total']
#     total_called = len(parsed)
    df = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
    while current_offset < (num+50) and current_offset < 1000:     
#         concat or append df here
#             current_offset += 50
#             url_params['offset'] = current_offset
#             results = yelp_call(headers, url_params)
#             parsed = parse_data(results['businesses'])
            
            df2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
            df.append(df2)
        current_offset += 50
        url_params['offset'] = current_offset
        results = yelp_call(headers, url_params)
        parsed = parse_data(results['businesses'])
    return df


#     pd.concat([df, df_2])

# df_save(csv_filepath, df)
# pd.read_csv(csv_filepath)
# Jake's suggestion, create workaround that checks to see —— 
#if ['businesses'] in data.keys():